<a href="https://colab.research.google.com/github/rpfilomeno/JupiterNotebooks/blob/main/Whisper_STT_and_Coqui_TTS_VIDEO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fine tune a VITS model with the Coqui TTS framework using audio samples of your choice.**

Thank you to all of the [Coqui TTS](https://https://github.com/coqui-ai/TTS) contributors, the OpenAI team,and [@Thorsten-Voice](https://https://www.youtube.com/c/ThorstenMueller ) (I think he wrote this original script). I have no idea who wrote the rnnoise loop, but they are great, too. I just smash things together. [-nn](https://https://www.youtube.com/c/NanoNomad )

If you have run the script before, and want to examine your training session using Tensorboard, set your dataset directories, then skip to the bottom section to load Tensorboard.

Run this cell to connect your Google Drive account to save files.

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


Set dataset and trainer output directory (model states saved here, subdirectory of dataset). Set upload_dir and place all audio files (Wav, mp3) in upload directory which will be /content/drive/MyDrive/{upload_dir}

In [ ]:
dataset_name = "me2-dataset" #@param {type:"string"}
output_directory = "traineroutput" #@param {type:"string"}
upload_dir = "mp3uploads" #@param {type:"string"}
upload_dir = "/content/drive/MyDrive/" + upload_dir
!mkdir $upload_dir

mkdir: cannot create directory ‘/content/drive/MyDrive/mp3uploads’: File exists


Cleanup previous audio processing

In [ ]:
!rm -rf /content/drive/MyDrive/$upload_dir/out
!rm -rf /content/drive/MyDrive/$dataset_name/converted

Download and Build Rnnoise (https://github.com/xiph/rnnoise) and Requirements (Optional if not processing .wav files for a new dataset. Multiple passes of rnnoise over samples may degrade audio quality. You should also preview your output dataset before training to ensure that the processing has not degraded any of your samples.  Occasionally, it will denoise too much of the tail end of phrases)

In [ ]:
!pip install pyloudnorm
!git clone https://github.com/xiph/rnnoise.git
!sudo apt-get install curl autoconf automake libtool python-dev pkg-config sox ffmpeg
%cd /content/rnnoise
!sh autogen.sh
!sh configure
!make clean
!make

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'rnnoise'...
remote: Enumerating objects: 420, done.
remote: Counting objects: 100% (219/219), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 420 (delta 194), reused 192 (delta 192), pack-reused 201
Receiving objects: 100% (420/420), 836.04 KiB | 12.67 MiB/s, done.
Resolving deltas: 100% (221/221), done.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
pkg-config is already the newest version (0.29.1-0ubuntu2).
python-dev is already the newest version (2.7.15~rc1-1).
curl is already the newest version (7.58.0-2ubuntu3.21).
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  autotools-dev file libmagic-mgc lib

Install Sox, Install OpenAI Whisper STT+Translation (https://github.com/openai/whisper)

In [ ]:
%cd /content
!sudo apt install sox
!git clone https://github.com/openai/whisper.git
!pip install git+https://github.com/openai/whisper.git 


/content
Reading package lists... Done
Building dependency tree       
Reading state information... Done
sox is already the newest version (14.4.2-3ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
Cloning into 'whisper'...
remote: Enumerating objects: 249, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 249 (delta 0), reused 2 (delta 0), pack-reused 244
Receiving objects: 100% (249/249), 3.13 MiB | 1.86 MiB/s, done.
Resolving deltas: 100% (141/141), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-cp469axz
  Running command git clone -q https://github.com/openai/whisper.git /tmp/pip-req-build-cp469axz
     |███████████████████████

Install Coqui TTS (https://github.com/coqui-ai/TTS), espeak-ng phonemeizer (https://github.com/espeak-ng/espeak-ng), download Coqui TTS source and examples from GitHub

In [ ]:
#@title
%cd /content
!sudo apt-get install espeak-ng
!git clone https://github.com/coqui-ai/TTS.git
!pip install TTS
!tts --list_models

/content
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
The following NEW packages will be installed:
  espeak-ng espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 20 not upgraded.
Need to get 3,957 kB of archives.
After this operation, 10.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libpcaudio0 amd64 1.0-1 [6,536 B]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsonic0 amd64 0.2.0-6 [13.4 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/main amd64 espeak-ng-data amd64 1.49.2+dfsg-1 [3,469 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/main amd64 libespeak-ng1 amd64 1.49.2+dfsg

Convert mp3 files in upload_dir to wav as 22050hz mono wav
Place output wav files in directory named 'out'.
Copy any uploaded .wav files to 'out' as 22050hz mono wav.

Make directory 'splits'. Use sox to split wav files into 8 second segments, placed in 'splits'.

In [ ]:
#@title
%cd $upload_dir
!rm -rf $upload_dir/out
!mkdir $upload_dir/out
!find . -name '*.mp3' -exec bash -c 'for f; do ffmpeg -i "$f" -acodec pcm_s16le -ar 22050 -ac 1 out/"${f%.mp3}".wav ; done' _ {} +
!find . -name '*.wav' -exec bash -c 'for f; do ffmpeg -i "$f" -acodec pcm_s16le -ar 22050 -ac 1 out/"${f%.wav}".wav ; done' _ {} +


!ls -al $upload_dir/out
!mkdir $upload_dir/out/splits
%cd $upload_dir/out
!for FILE in *.wav; do sox "$FILE" splits/"$FILE" --show-progress silence 1 0.5 0.1% 1 0.5 0.1% : newfile : restart ; done
#alt split method: force splits of 8 seconds, however this will split words. Comment the above with # and remove the # below to change
#!for FILE in *.wav; do sox "$FILE" splits/"$FILE" --show-progress trim 0 8 : restart ; done
%cd $upload_dir/out/splits
!find . -name "*.wav" -type f -size -15k -delete

Streaming output truncated to the last 5000 lines.

In:100%  00:00:08.00 [00:00:00.00] Out:156k  [ -====|====- ]        Clip:0    
Done.

Input File     : 'dg534003.wav'
Channels       : 1
Sample Rate    : 22050
Precision      : 16-bit
Duration       : 00:00:08.00 = 176400 samples ~ 600 CDDA sectors
File Size      : 353k
Bit Rate       : 353k
Sample Encoding: 16-bit Signed Integer PCM

In:55.7% 00:00:04.46 [00:00:03.54] Out:28.6k [ =====|===== ]        Clip:0    
In:100%  00:00:08.00 [00:00:00.00] Out:103k  [ -====|====- ]        Clip:0    
Done.

Input File     : 'dg534004.wav'
Channels       : 1
Sample Rate    : 22050
Precision      : 16-bit
Duration       : 00:00:05.94 = 130977 samples ~ 445.5 CDDA sectors
File Size      : 262k
Bit Rate       : 353k
Sample Encoding: 16-bit Signed Integer PCM

In:100%  00:00:05.94 [00:00:00.00] Out:78.5k [  ====|====  ]        Clip:0    
Done.

Input File     : 'dh001001.wav'
Channels       : 1
Sample Rate    : 22050
Precision      : 16-bit
Duration 

(Optional) Run rnnoise, processed clips will be saved to your Google Drive in a folder named 'converted'. Files converted to 22khz mono for VITS model fine tuning.

Original author unknown, but thank you for posting this helpful code.

In [ ]:
#@title
orig_wavs= upload_dir + "/out/splits"
from pathlib import Path
import os
import subprocess
import soundfile as sf
import pyloudnorm as pyln
import sys
import glob
rnn = "/content/rnnoise/examples/rnnoise_demo"
paths = glob.glob(os.path.join(orig_wavs, '*.wav'))
for filepath in paths:
  base = os.path.basename(filepath)
  tp_s = "/content/drive/MyDrive/" + dataset_name + "/" + "converted" + "/"
  tf_s = "/content/drive/MyDrive/" + dataset_name + "/" + "converted" + "/" + base
  target_path = Path(tp_s)
  target_file = Path(tf_s)
  print("From: " + str(filepath))
  print("To: " + str(target_file))
	
# Stereo to Mono; upsample to 48000Hz
# added -G to fix gain, -v 0.95
  subprocess.run(["sox", "-G", "-v", "0.95", filepath, "48k.wav", "remix", "-", "rate", "48000"])
  subprocess.run(["sox", "48k.wav", "-c", "1", "-r", "48000", "-b", "16", "-e", "signed-integer", "-t", "raw", "temp.raw"]) # convert wav to raw
  subprocess.run(["/content/rnnoise/examples/rnnoise_demo", "temp.raw", "rnn.raw"]) # apply rnnoise
  subprocess.run(["sox", "-G", "-v", "0.95", "-r", "48k", "-b", "16", "-e", "signed-integer", "rnn.raw", "-t", "wav", "rnn.wav"]) # convert raw back to wav

  subprocess.run(["mkdir", "-p", str(target_path)])
  subprocess.run(["sox", "rnn.wav", str(target_file), "remix", "-", "highpass", "100", "lowpass", "7000", "rate", "22050"]) # apply high/low pass filter and change sr to 22050Hz
  data, rate = sf.read(target_file)

# peak normalize audio to -1 dB
  peak_normalized_audio = pyln.normalize.peak(data, -1.0)

# measure the loudness first
  meter = pyln.Meter(rate) # create BS.1770 meter
  loudness = meter.integrated_loudness(data)

# loudness normalize audio to -25 dB LUFS
  loudness_normalized_audio = pyln.normalize.loudness(data, loudness, -25.0)
  sf.write(target_file, data=loudness_normalized_audio, samplerate=22050)
  print("")
%cd /content/drive/MyDrive/"$dataset_name"/converted/
!find . -name "*.wav" -type f -size -15k -delete


From: /content/drive/MyDrive/mp3uploads/out/splits/2sample10001001.wav
To: /content/drive/MyDrive/me2-dataset/converted/2sample10001001.wav

From: /content/drive/MyDrive/mp3uploads/out/splits/2sample1001001.wav
To: /content/drive/MyDrive/me2-dataset/converted/2sample1001001.wav

From: /content/drive/MyDrive/mp3uploads/out/splits/2sample11001001.wav
To: /content/drive/MyDrive/me2-dataset/converted/2sample11001001.wav

From: /content/drive/MyDrive/mp3uploads/out/splits/2sample12001001.wav
To: /content/drive/MyDrive/me2-dataset/converted/2sample12001001.wav

From: /content/drive/MyDrive/mp3uploads/out/splits/2sample13001001.wav
To: /content/drive/MyDrive/me2-dataset/converted/2sample13001001.wav

From: /content/drive/MyDrive/mp3uploads/out/splits/2sample14001001.wav
To: /content/drive/MyDrive/me2-dataset/converted/2sample14001001.wav

From: /content/drive/MyDrive/mp3uploads/out/splits/2sample14001002.wav
To: /content/drive/MyDrive/me2-dataset/converted/2sample14001002.wav

From: /content/

Copy wav files to dataset wav directory

In [ ]:
#@title
!mkdir /content/drive/MyDrive/$dataset_name/wavs
!cp /content/drive/MyDrive/$dataset_name/converted/*.wav /content/drive/MyDrive/$dataset_name/wavs
!ls -al /content/drive/MyDrive/$dataset_name/wavs

mkdir: cannot create directory ‘/content/drive/MyDrive/me2-dataset/wavs’: File exists
total 19235
-rw------- 1 root root  46790 Dec 19 08:16 '2-04 - Mark  The Resurrection of the Christ (16)001.wav'
-rw------- 1 root root 374452 Dec 19 08:16 '2-04 - Mark  The Resurrection of the Christ (16)002.wav'
-rw------- 1 root root 231568 Dec 19 08:16 '2-04 - Mark  The Resurrection of the Christ (16)003.wav'
-rw------- 1 root root 240388 Dec 19 08:16 '2-04 - Mark  The Resurrection of the Christ (16)004.wav'
-rw------- 1 root root 213928 Dec 19 08:16 '2-04 - Mark  The Resurrection of the Christ (16)005.wav'
-rw------- 1 root root 337850 Dec 19 08:16 '2-04 - Mark  The Resurrection of the Christ (16)006.wav'
-rw------- 1 root root  50318 Dec 19 08:16 '2-04 - Mark  The Resurrection of the Christ (16)007.wav'
-rw------- 1 root root 202904 Dec 19 08:16 '2-04 - Mark  The Resurrection of the Christ (16)008.wav'
-rw------- 1 root root  36206 Dec 19 08:16 '2-04 - Mark  The Resurrection of the Christ (16)00

Run Whisper on generated audio clips, generate transcript named metadata.csv in LJSpeech format in the dataset directory.

In [ ]:
#@title
import whisper
import os, os.path
import glob
import pandas as pd

from pathlib import Path

wavs = '/content/drive/MyDrive/'+dataset_name+'/wavs'

model = whisper.load_model("medium.en")
paths = glob.glob(os.path.join(wavs, '*.wav'))
print(len(paths))

all_filenames = []
transcript_text = []
with open('/content/drive/MyDrive/'+dataset_name+'/metadata.csv', 'w', encoding='utf-8') as outfile:
	for filepath in paths:
		base = os.path.basename(filepath)
		all_filenames.append(base)
	for filepath in paths:
		result = model.transcribe(filepath)
		output = result["text"].lstrip()
		output = output.replace("\n","")
		thefile = str(os.path.basename(filepath).lstrip(".")).rsplit(".")[0]
		outfile.write(thefile + '|' + output + '|' + output + '\n')
		print(thefile + '|' + output + '|' + output + '\n')


Display dataset

In [ ]:
#@title
!cat /content/drive/MyDrive/$dataset_name/metadata.csv

2-04 - Mark  The Resurrection of the Christ (16)001|Chapter 16.|Chapter 16.
2-04 - Mark  The Resurrection of the Christ (16)002|Now when the Sabbath was passed, Mary Magdalene, Mary the mother of James, and Salome bought spices that they might come and annoy him.|Now when the Sabbath was passed, Mary Magdalene, Mary the mother of James, and Salome bought spices that they might come and annoy him.
2-04 - Mark  The Resurrection of the Christ (16)003|Very early in the morning, on the first day of the week, they came to the tomb when the sun had risen.|Very early in the morning, on the first day of the week, they came to the tomb when the sun had risen.
2-04 - Mark  The Resurrection of the Christ (16)004|And they said among themselves, who will roll away the stone from the door of the tomb for us?|And they said among themselves, who will roll away the stone from the door of the tomb for us?
2-04 - Mark  The Resurrection of the Christ (16)005|But when they looked up, they saw that the stone

Download VITS model and Generate Sample Wav File to /content/ljspeech-vits.wav  This will be deleted when your Colab session is closed.

In [ ]:
!tts --text "I am the very model of a modern Major General" --model_name "tts_models/en/ljspeech/vits" --out_path /content/ljspeech-vits.wav

 > Downloading model to /root/.local/share/tts/tts_models--en--ljspeech--vits
100% 146M/146M [00:15<00:00, 9.34MiB/s]
 > Model's license - apache 2.0
 > Check https://choosealicense.com/licenses/apache-2.0/ for more info.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Text: I a

Save the training script with your dataset name, and output dir set. File will be saved in your Google drive dataset folder as train_vits.py

In [ ]:
#@title
code = """import os

from trainer import Trainer, TrainerArgs

from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.vits import Vits, VitsAudioConfig
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

#output_path = os.path.dirname(os.path.abspath(__file__))
##########################################
#Change this to your dataset directory
##########################################
output_path = "/content/drive/MyDrive/"""
code = code + dataset_name + "/" + output_directory + "/" + "\""

code=code + """
dataset_config = BaseDatasetConfig(
##########################################
#Change this to your dataset directory
##########################################
    formatter="ljspeech", meta_file_train="metadata.csv", path=os.path.join(output_path, "/content/drive/MyDrive/"""
code = code + dataset_name
code=code + """")

)
audio_config = VitsAudioConfig(
    sample_rate=22050, win_length=1024, hop_length=256, num_mels=80, mel_fmin=0, mel_fmax=None
)

config = VitsConfig(
    audio=audio_config,
    run_name="vits_ljspeech",
    batch_size=12,
    eval_batch_size=12,
    batch_group_size=4,
#    num_loader_workers=8,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=100000,
    save_step=1000,
	  save_checkpoints=True,
	  save_n_checkpoints=4,
	  save_best_after=1000,
    #text_cleaner="english_cleaners",
    text_cleaner="multilingual_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    compute_input_seq_cache=True,
    print_step=25,
    print_eval=True,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    cudnn_benchmark=False,
)

# INITIALIZE THE AUDIO PROCESSOR
# Audio processor is used for feature extraction and audio I/O.
# It mainly serves to the dataloader and the training loggers.
ap = AudioProcessor.init_from_config(config)

# INITIALIZE THE TOKENIZER
# Tokenizer is used to convert text to sequences of token IDs.
# config is updated with the default characters if not defined in the config.
tokenizer, config = TTSTokenizer.init_from_config(config)

# LOAD DATA SAMPLES
# Each sample is a list of ```[text, audio_file_path, speaker_name]```
# You can define your custom sample loader returning the list of samples.
# Or define your custom formatter and pass it to the `load_tts_samples`.
# Check `TTS.tts.datasets.load_tts_samples` for more details.
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

# init model
model = Vits(config, ap, tokenizer, speaker_manager=None)

# init the trainer and begin
trainer = Trainer(
    TrainerArgs(),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
)
trainer.fit()
"""
#print(code)
myFile = open("/content/drive/MyDrive/" + dataset_name + "/train_vits.py", 'w')

myFile.write(code)
myFile.close()
%ls -al /content/drive/MyDrive/$dataset_name/train_vits.py

-rw------- 1 root root 2995 Dec 19 08:26 /content/drive/MyDrive/me2-dataset/train_vits.py


Fine Tune VITS model

In [ ]:
!CUDA_VISIBLE_DEVICES="0" python /content/drive/MyDrive/$dataset_name/train_vits.py --restore_path /root/.local/share/tts/tts_models--en--ljspeech--vits/model_file.pth --config_file /root/.local/share/tts/tts_models--en--ljspeech--vits/config.json

**Resume a fine tuning session**

Run the next cell to list all of the saved run direcories.  Copy select the run you want to resume, ctrl+C to copy the name. Paste in the next cell and run it.

In [ ]:
!ls /content/drive/MyDrive/$dataset_name/$output_directory

phoneme_cache
vits_ljspeech-December-19-2022_08+27AM-0000000
vits_ljspeech-December-19-2022_08+32AM-0000000
vits_ljspeech-December-19-2022_08+33AM-0000000
vits_ljspeech-December-19-2022_08+34AM-0000000
vits_ljspeech-December-19-2022_08+37AM-0000000


In [ ]:
run_name = "vits_ljspeech-December-19-2022_08+37AM-0000000" #@param {type:"string"}

Run this cell to list all of the checkpoints in the saved run, then put the checkpoint name in the next cell, and run it.

In [ ]:
!ls /content/drive/MyDrive/$dataset_name/$output_directory/$run_name

best_model_1000011.pth	events.out.tfevents.1671439076.90a9942a32ea
best_model.pth		trainer_0_log.txt
config.json		train_vits.py


In [ ]:
model_checkpoint = "best_model_1000011.pth" #@param {type:"string"}

Run the next cell to restore a previous fine tuning session using your dataset name, trainer output directory, and model checkpoint set

In [ ]:
!CUDA_VISIBLE_DEVICES="0" python /content/drive/MyDrive/$dataset_name/train_vits.py --restore_path /content/drive/MyDrive/$dataset_name/$output_directory/$run_name/$model_checkpoint --config_path /content/drive/MyDrive/$dataset_name/$run_name/config.json

View Memory Usage

In [ ]:
!nvidia-smi

Wed Nov 30 00:54:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Load Tensorboard

In [ ]:
import torch 
%load_ext tensorboard

Load Dashboard. May take several minutes to appear from a blank white box.  Ad blockers probably need to whitelist a bunch of Colab stuff or this won't wrok.

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/$dataset_name/$output_directory/